In [4]:
import pandas as pd
import numpy as np 
import os 
import psycopg2
import shutil

In [5]:
# Find csv files in the directory
# isolating the csv file
csv_file = []

for file in os.listdir(os.path.join(os.getcwd(), 'datasets')):
    if file.endswith('.csv'):
        csv_file.append(file)

In [6]:
csv_ds_path = os.path.join(os.getcwd(), 'csv_ds')
if not os.path.isdir(csv_ds_path):
    os.mkdir(csv_ds_path)

for csv in csv_file:
    src = os.path.join(os.getcwd(), 'datasets', csv)
    dst = os.path.join(csv_ds_path, csv)
    shutil.copy(src, dst)
    print(f'Moved {csv} to {csv_ds_path}')

Moved Customer Contracts$.csv to c:\Users\aariz\codes\snippetCollection\python\Automate_daily_task\automating_import_CSV_files_to_database\csv_ds
Moved Customer Demo.csv to c:\Users\aariz\codes\snippetCollection\python\Automate_daily_task\automating_import_CSV_files_to_database\csv_ds
Moved Customer Engagements.csv to c:\Users\aariz\codes\snippetCollection\python\Automate_daily_task\automating_import_CSV_files_to_database\csv_ds


#### Create the pandas df from the csv file

In [76]:
data_path = os.path.join(os.getcwd(), 'csv_ds')

df = {}

for file in csv_file:
    try:
        df[file] = pd.read_csv(os.path.join(data_path, file))
    except UnicodeDecodeError:
        # encoding='ISO-8859-1', you are telling pandas to use the ISO-8859-1 encoding to read the file, 
        # which can handle a different set of characters
        df[file] = pd.read_csv(os.path.join(data_path, file), encoding='ISO-8859-1')

In [77]:
for k in csv_file:
    dataframe = df[k]
    
    clean_tbl_name = k.lower().replace(' ', '_').replace('?', '') \
                             .replace('-', '_').replace(r'/', '_').replace('\\', '_').replace('%', '') \
                             .replace(')', '').replace(r'(', '').replace('$', '')
    # remove the .csv extension 
    
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    # print(tbl_name)
    dataframe.columns = [x.lower().replace(' ', '_').replace('?', '') \
                             .replace('-', '_').replace(r'/', '_').replace('\\', '_').replace('%', '') \
                             .replace(')', '').replace(r'(', '').replace('$', '') for x in dataframe.columns]
    
    # replacemet dictionary that maps pandas dtypes to sql dtypes
    replacements = {
    'object'          : 'varchar',
    'float64'         : 'float',
    'int64'           : 'int',
    'datetime64'      : 'timestamp',
    'timedelta64[ns]' : 'varchar'
    }

    # table schema
    col_str = ', '.join('{} {}'.format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    try:
        conn = psycopg2.connect(user="aariz", password="2Atrium@cortex#8", host="dbserverpg.postgres.database.azure.com", port=5432, database="postgres")
        cursor = conn.cursor()
        print('Opened Database successfully')
    except psycopg2.OperationalError as e:  
        print(f"Error: {e}")
        
    cursor.execute('DROP TABLE IF EXISTS {};'.format(tbl_name))
    
    # creating table
    cursor.execute('CREATE TABLE {} ({});'.format(tbl_name, col_str))
    print('{0} was created successfully '.format(tbl_name))
    
    # insert values to the table

    # save df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')

    # we are opening the df in our ram as a Object 
    my_file = open(k)
    print('file opened in memory')
        
    SQL_STATEMENT = '''
    COPY %s FROM STDIN WITH 
        CSV 
        HEADER 
        DELIMITER AS ','
    '''

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name ,file=my_file)
    print('file copied to the db')
    
    cursor.execute('grant select on table %s to public' % tbl_name)
    conn.commit()
    conn.close()
    print('table {0} imported to db completed'.format(tbl_name))
    
print('all tables has been successfully been imported to the db')

Opened Database successfully
customer_contracts was created successfully 
file opened in memory
file copied to the db
table customer_contracts imported to db completed
Opened Database successfully
customer_demo was created successfully 
file opened in memory
file copied to the db
table customer_demo imported to db completed
Opened Database successfully
customer_engagements was created successfully 
file opened in memory
file copied to the db
table customer_engagements imported to db completed
Opened Database successfully
customer_contracts was created successfully 
file opened in memory
file copied to the db
table customer_contracts imported to db completed
all tables has been successfully been imported to the db


In [78]:
# Ensure the connection is open
if conn.closed:
    conn = psycopg2.connect(user="*****", password="**********", host="dbserverpg.postgres.database.azure.com", port=5432, database="postgres")

# Create a new cursor
cursor = conn.cursor()

cursor.execute("SELECT datname FROM pg_database;")
databases = cursor.fetchall()
print("Databases:")
for db in databases:
    print(f"- {db[0]}")

Databases:
- azure_maintenance
- template1
- postgres
- azure_sys
- template0


In [79]:
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
""")
tables = cursor.fetchall()

for table in tables:
    print(table[0])

customer_demo
customer_engagements
customer_contracts


In [80]:
cursor.close()